# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [273]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import  word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import PorterStemmer
import pickle


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrador\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrador\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Administrador\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [215]:
# load data from database
engine = create_engine('sqlite:///Data/Disaster.db')
df = pd.read_sql_table('T_Responses', engine)
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [216]:
def tokenize(text):
    '''
    Thif function tokenizes a text after doing some data wrangling

    1. Sets all the text in lower case
    2. Removes punctuation
    3. Tokenizes the text 
    4 Remove stop words
    5. Does lemmatization of the words
    6. Does stemmization of the words
    
    '''
     #Lower case
    text = text.lower()
    #Removes puntuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    #Tokenizes
    words = word_tokenize(text)
    #Removes stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    #Lemmatizes
    words = [WordNetLemmatizer().lemmatize(w) for w in words]
    #Stemmizes
    words= [PorterStemmer().stem(w) for w in words]

    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [239]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [263]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, test_size = 0.30)
pipeline_fitted = pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [264]:
y_prediction_test = pipeline_fitted.predict(X_test)
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.85      0.94      0.90      5992
               request       0.84      0.48      0.61      1381
                 offer       0.00      0.00      0.00        35
           aid_related       0.77      0.68      0.72      3317
          medical_help       0.65      0.07      0.12       636
      medical_products       0.80      0.09      0.16       413
     search_and_rescue       0.86      0.05      0.10       219
              security       0.50      0.01      0.01       140
              military       0.75      0.04      0.07       242
                 water       0.88      0.37      0.53       481
                  food       0.87      0.56      0.68       872
               shelter       0.85      0.35      0.49       709
              clothing       0.67      0.15      0.25       119
                 money       1.00      0.01      0.02       189
        missing_people       1.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [249]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

In [266]:
parameters = {
        'clf__estimator__max_depth': [ 80, 90, 100],
        'clf__estimator__n_estimators': [5, 10, 15]}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

In [267]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] clf__estimator__max_depth=80, clf__estimator__n_estimators=5 ....
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  clf__estimator__max_depth=80, clf__estimator__n_estimators=5, score=0.182, total= 2.5min
[CV] clf__estimator__max_depth=80, clf__estimator__n_estimators=5 ....
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s
[CV]  clf__estimator__max_depth=80, clf__estimator__n_estimators=5, score=0.193, total= 2.2min
[CV] clf__estimator__max_depth=80, clf__estimator__n_estimators=5 ....
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.7min remaining:    0.0s
[CV]  clf__estimator__max_depth=80, clf__estimator__n_estimators=5, score=0.189, total= 2.1min
[CV] clf__estimator__max_depth=80, clf__estimator__n_estimators=5 ....
[CV]  clf__estimator__max_depth=80, clf__estimator__n_estimators=5, score=0.193, total= 2.2min
[CV] clf__estimator__max_depth=80, 

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001F98B4A19D0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_depth': [80, 90, 100],
                         'clf__estimator__n_estimators': [5, 10, 15]},
             verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [268]:
y_prediction_test = cv.predict(X_test)
print(classification_report(y_test.values, y_prediction_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.80      0.98      0.88      5992
               request       0.84      0.41      0.55      1381
                 offer       0.00      0.00      0.00        35
           aid_related       0.77      0.60      0.68      3317
          medical_help       0.57      0.07      0.12       636
      medical_products       0.80      0.07      0.12       413
     search_and_rescue       0.75      0.04      0.08       219
              security       0.00      0.00      0.00       140
              military       0.74      0.11      0.19       242
                 water       0.84      0.26      0.39       481
                  food       0.89      0.39      0.54       872
               shelter       0.85      0.23      0.36       709
              clothing       0.72      0.24      0.36       119
                 money       0.50      0.02      0.03       189
        missing_people       1.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [270]:
cv.best_params_

{'clf__estimator__max_depth': 100, 'clf__estimator__n_estimators': 15}

### 9. Export your model as a pickle file

In [274]:
pickle.dump(cv.best_estimator_, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.